# Portfolio Sorts

This notebook implements a simple portfolio sort.

## Load Packages and Extra Functions

In [1]:
using Printf, Dates, DelimitedFiles, Statistics

include("jlFiles/printmat.jl")
include("jlFiles/lagFn.jl")                #creating lags of a matrix
include("jlFiles/EMAFn.jl")                #moving average of rows of a matrix

include("jlFiles/sorts.jl")

   0   1   0   1   0   0
   0   1   1   0   0   0

   1   0   1   0   0   0
   1   0   0   1   0   0

   0   0   0   0   0   0
   0   0   0   0   0   0

   0   0   0   0   0   0
   0   0   0   0   0   0



## Load Data

The data set contains daily data for "dates", the equity market return, riskfree rate and the the returns of the 25 Fama-French portfolios. All returns are in percent.

In [2]:
x   = readdlm("Data/MomentumSR.csv",',')
dN  = Date.(x[:,1],"yyyy-mm-dd")                  #Julia dates 
y   = convert.(Float64,x[:,2:end])

(Rm,Rf,R) = (y[:,1],y[:,2],y[:,3:end])

println("\nThe first few rows of dN, Rm and Rf")
printmat([dN[1:4] Rm[1:4] Rf[1:4]])

println("size of dN, Rm, Rf, R")
println(size(dN),"\n",size(Rm),"\n",size(Rf),"\n",size(R))

(T,n) = size(R);                      #number of periods and assets


The first few rows of dN, Rm and Rf
1979-01-02     0.615     0.035
1979-01-03     1.155     0.035
1979-01-04     0.975     0.035
1979-01-05     0.685     0.035

size of dN, Rm, Rf, R
(9837,)
(9837,)
(9837,)
(9837, 25)


## Creating Signals as Moving Averages

In [3]:
vol = EMAFn(abs.(R),22)                       #MA(22) of |R|
ret = EMAFn(R,22)                             #MA(22) of R

Z1 = lagFn(vol)                                    #sort on vol[t-1]
Z2 = lagFn(ret)                                    #sort on ret[t-1]

println()

## Implementing the Portfolio Sorts

1.  Sort  $R_{t-1}$  across the 25 assets.
2. (In the evening of) period  $t-1$, construct the `Hi` portfolio by buying 1/5 of each of the 5 best assets based on the sort in point 1. Similarly, construct the `Lo` portfolio by buying 1/5 of each of the 5 worst assets. Collect these portfolio weights in the vectors `wHi` and `wLo`.
3. In period  $t$ , the return on the `Hi` portfolio is $R_{p,t}=wHi'R_{t}$. Similarly for `Lo`.
4. Repeat for all periods

In [4]:
(RHi,RLo) = (fill(NaN,T),fill(NaN,T))
for t = 2:T         #loop over periods, save portfolio returns
    #local sort1, wHi, wLo          #only needed in script
    sort1                  = sortperm(R[t-1,:])  #sort1[1] is index of worst asset
    (wLo,wHi)              = (zeros(n),zeros(n))
    wLo[sort1[1:5]]       .= 1/5    #equally weighted inside Lo portfolio
    wHi[sort1[end-4:end]] .= 1/5
    RLo[t]                 = wLo'R[t,:]
    RHi[t]                 = wHi'R[t,:]
end

ReLo = RLo[2:end] - Rf[2:end]
ReHi = RHi[2:end] - Rf[2:end];       #cut out t=1, excess returns

Calculate the mean (excess) return, its standard deviation and the Sharpe ratio. Annualize by assuming 252 trading days per year. Compare with the excess return on passively holding an equity market index.

In [5]:
```

Calculate average excess return, the std and the SR ratio - and annualise all of them.
```
function ReturnStats(Re,Annfactor=252)
    μ  = mean(Re)*Annfactor
    σ  = std(Re)*sqrt(Annfactor)
    SR = μ/σ
    stats = [μ,σ,SR]
    return stats
end

ReturnStats (generic function with 2 methods)

In [6]:
Rme = Rm - Rf

StatLo = ReturnStats(ReLo,252)
StatHi = ReturnStats(ReHi,252)
Statm  = ReturnStats(Rme[2:end],252)

printblue("Stats for the portfolio returns, annualized:")
printmat([StatLo StatHi Statm],colNames=["Lo" "Hi" "market"],rowNames=["μ";"σ";"SR"])

Stats for the portfolio returns, annualized:
          Lo        Hi    market
μ      1.651    17.914     8.374
σ     18.706    17.610    16.837
SR     0.088     1.017     0.497



## Univariate Sort on Past Returns

In [7]:
L = 22       #MA over L periods to create signals

Z1 = lagFn(EMAFn(R,L));

In [8]:
M1 = 5                                  #number of assets in each portfolio

(GLo,GHi) = PortfSortLoHiPs(Z1,M1)      #Txn bitarrays indicating group membership

println("printing the first few lines of GLo and GHi")
printmat(GLo[1:5,:],prec=0,width=3)
printmat(GHi[1:5,:],prec=0,width=3)

printred("do not use the first rows sinze Z1[t,:] is NaN")

printing the first few lines of GLo and GHi
  1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0  1  0  0  0  0  0  1  0  0
  0  0  0  0  0  0  0  1  1  0  0  0  0  1  0  0  1  0  0  0  0  0  1  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  1  1  1  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  1  0  1  1  0

  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1
  0  0  0  0  1  0  0  0  0  0  0  1  0  0  1  0  0  0  0  1  0  0  0  0  1
  0  0  0  0  1  0  0  0  0  0  0  1  0  0  1  0  0  0  0  1  0  0  0  0  1
  0  0  0  0  1  0  1  0  0  0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  1
  1  0  0  0  1  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  1

do not use the first rows sinze Z1[t,:] is NaN


In [9]:
WLo = GLo./sum(GLo,dims=2)           #create portfolio weights
WHi = GHi./sum(GHi,dims=2)

RLo = sum(WLo.*R,dims=2)            #returns on the portfolios
RHi = sum(WHi.*R,dims=2)

ReLo = RLo[2:end] - Rf[2:end]
ReHi = RHi[2:end] - Rf[2:end];       #cut out t=1, excess returns

In [10]:
Rme = Rm - Rf

StatLo = ReturnStats(ReLo,252)
StatHi = ReturnStats(ReHi,252)
Statm  = ReturnStats(Rme[2:end],252)

printblue("Stats for the portfolio returns, annualized:")
printmat([StatLo StatHi Statm],colNames=["Lo" "Hi" "market"],rowNames=["μ";"σ";"SR"])

Stats for the portfolio returns, annualized:
          Lo        Hi    market
μ      3.836    14.168     8.374
σ     19.042    17.331    16.837
SR     0.201     0.818     0.497



In [11]:
M1 = 5


(RHi,RLo) = (fill(NaN,T),fill(NaN,T))
for t = 2:T         #loop over periods, save portfolio returns
    #local sort1, wHi, wLo          #only needed in script
    sort1                  = sortperm(R[t-1,:])  #sort1[1] is index of worst asset
    (wLo,wHi)              = (zeros(n),zeros(n))
    wLo[sort1[1:M1]]         .= 1/M1    #equally weighted inside Lo portfolio
    wHi[sort1[end-M1+1:end]] .= 1/M1
    RLo[t]                 = wLo'R[t,:]
    RHi[t]                 = wHi'R[t,:]
end

ReLo = RLo[2:end] - Rf[2:end]
ReHi = RHi[2:end] - Rf[2:end];       #cut out t=1, excess returns

Rme = Rm - Rf

StatLo = ReturnStats(ReLo,252)
StatHi = ReturnStats(ReHi,252)
Statm  = ReturnStats(Rme[2:end],252)

printblue("Stats for the portfolio returns, annualized:")
printmat([StatLo StatHi Statm],colNames=["Lo" "Hi" "market"],rowNames=["μ";"σ";"SR"])

Stats for the portfolio returns, annualized:
          Lo        Hi    market
μ      1.651    17.914     8.374
σ     18.706    17.610    16.837
SR     0.088     1.017     0.497



In [23]:
using StatsBase

L = 22
Z1 = lagFn(EMAFn(abs.(R),L))
Z2 = lagFn(EMAFn(R,L))

M1 = 10
M2 = 5

t = 100
sort1           = sortperm(Z1[t,:])  #sort1[1] is index of the asset with lowest Z1
(sort2L,sort2H) = (falses(n),falses(n))   
sort2L[sort1[1:M1]]         .= true
sort2H[sort1[end-M1+1:end]] .= true

printmat(1:n,Z1[t,:],ordinalrank(Z1[t,:]),sort2L,sort2H)

error()
(RHi,RLo) = (fill(NaN,T),fill(NaN,T))
for t = 2:T         #loop over periods, save portfolio returns
    #local sort1, wHi, wLo          #only needed in script
    sort1           = sortperm(Z1[t,:])  #sort1[1] is index of worst asset

    sort2
    
    (wL,wH)                = (zeros(n),zeros(n))
    wL[sort1[1:M1]]         .= 1/M1    #equally weighted inside Lo portfolio
    wH[sort1[end-M1+1:end]] .= 1/M1
    RLo[t]                 = wL'R[t,:]
    RH[t]                 = wH'R[t,:]
end

ReLo = RLo[2:end] - Rf[2:end]
ReHi = RHi[2:end] - Rf[2:end];       #cut out t=1, excess returns

Rme = Rm - Rf

StatLo = ReturnStats(ReLo,252)
StatHi = ReturnStats(ReHi,252)
Statm  = ReturnStats(Rme[2:end],252)

printblue("Stats for the portfolio returns, annualized:")
printmat([StatLo StatHi Statm],colNames=["Lo" "Hi" "market"],rowNames=["μ";"σ";"SR"])

     1.000     0.642    23.000     0.000     1.000
     2.000     0.468    12.000     0.000     0.000
     3.000     0.438     8.000     1.000     0.000
     4.000     0.408     3.000     1.000     0.000
     5.000     0.425     6.000     1.000     0.000
     6.000     0.660    24.000     0.000     1.000
     7.000     0.490    14.000     0.000     0.000
     8.000     0.454    11.000     0.000     0.000
     9.000     0.390     2.000     1.000     0.000
    10.000     0.513    18.000     0.000     1.000
    11.000     0.496    16.000     0.000     1.000
    12.000     0.595    22.000     0.000     1.000
    13.000     0.411     4.000     1.000     0.000
    14.000     0.449    10.000     1.000     0.000
    15.000     0.670    25.000     0.000     1.000
    16.000     0.565    21.000     0.000     1.000
    17.000     0.344     1.000     1.000     0.000
    18.000     0.431     7.000     1.000     0.000
    19.000     0.423     5.000     1.000     0.000
    20.000     0.556    20.000 

LoadError: 

In [15]:
sort1

25-element Array{Int64,1}:
 17
  9
  4
 13
 19
  5
 18
  3
 23
 14
  8
  2
 21
  7
 24
 11
 25
 10
 22
 20
 16
 12
  1
  6
 15